In [3]:
import pandas as pd

df_train = pd.read_csv('dataset/df_train_merged.csv')
df_test = pd.read_csv('dataset/test.csv')
df_holdout = pd.read_csv('dataset/holdout.csv', delimiter=';')

df_train.head()

,comment,label
0,aamiin ya rabb,0
1,terima kasih mengajak jalan2 virtual raja ampa...,0
2,bener prabu,0
3,tonton video ya hehe,0
4,coach nova plis suruh pda bljr sepak penalti a...,0


In [ ]:
# Pipeline feature engineering
# add another column: text length, count word, num emoji, num foreign character

import emoji

def count_emojis(text: str) -> int:
    if not isinstance(text, str):
        return 0
    # emoji.emoji_list returns a list of detected emoji segments
    return len(emoji.emoji_list(text))


def count_foreign_alpha_characters(text: str) -> int:
    """
    Count alphabetic characters that are non-ASCII, e.g., mathematical bold/italic letters like 𝘿, 𝐸, 𝐖, etc.
    """
    if not isinstance(text, str):
        return 0
    count = 0
    for ch in text:
        if ch.isalpha() and ord(ch) > 127:
            count += 1
    return count


def compute_text_features(df, text_col: str = "comment"):
    if text_col not in df.columns:
        return df
    s = df[text_col].fillna("")
    df["text_length"] = s.str.len()
    df["count_word"] = s.str.split().str.len()
    df["num_emoji"] = s.apply(count_emojis)
    df["num_foreign_character"] = s.apply(count_foreign_alpha_characters)
    return df

# Apply to available datasets
for _df_name in ["df_train", "df_test", "df_holdout"]:
    if _df_name in globals():
        _df = globals()[_df_name]
        globals()[_df_name] = compute_text_features(_df, text_col="comment")

# Preview
try:
    df_train.head()
except Exception:
    pass

In [10]:
# Save engineered datasets to disk
import os

output_dir = "dataset/feature_enginering"
os.makedirs(output_dir, exist_ok=True)

name_to_filename = {
    "df_train": "train_feature_engineered.csv",
    "df_test": "test_feature_engineered.csv",
    "df_holdout": "holdout_feature_engineered.csv",
}

for _df_name, _filename in name_to_filename.items():
    if _df_name in globals():
        _df = globals()[_df_name]
        _path = os.path.join(output_dir, _filename)
        _df.to_csv(_path, index=False)

# Show saved file list
try:
    sorted(os.listdir(output_dir))
except Exception:
    pass


In [33]:
def compute_text_features(text_col):
    def count_emojis(text: str) -> int:
        if not isinstance(text, str):
            return 0
        # emoji.emoji_list returns a list of detected emoji segments
        return len(emoji.emoji_list(text))

    def count_foreign_alpha_characters(text: str) -> int:
        """
        Count alphabetic characters that are non-ASCII, e.g., mathematical bold/italic letters like 𝘿, 𝐸, 𝐖, etc.
        """
        if not isinstance(text, str):
            return 0
        count = 0
        for ch in text:
            if ch.isalpha() and ord(ch) > 127:
                count += 1
        return count

    # If input is a string, compute features for a single string
    if isinstance(text_col, str):
        s = text_col
        result = {
            "text_length": len(s),
            "count_word": len(s.split()),
            "num_emoji": count_emojis(s),
            "num_foreign_character": count_foreign_alpha_characters(s),
        }
        return result
    # If input is a pandas Series, compute features for each row
    elif hasattr(text_col, "apply"):
        s = text_col.fillna("")
        result = {
            "text_length": s.str.len(),
            "count_word": s.str.split().str.len(),
            "num_emoji": s.apply(count_emojis),
            "num_foreign_character": s.apply(count_foreign_alpha_characters),
        }
        return result
    else:
        raise TypeError("Input must be a string or a pandas Series.")

In [32]:
print(compute_text_features("hello World"))

{'text_length': 11, 'count_word': 2, 'num_emoji': 0, 'num_foreign_character': 0}
